# Aspif pretty printing

The task of this notebook is to implement a pretty printer for the aspif format of clingo.

You can start from the script `aspif-pretty-printer.py`.

The classes `AspifPrinter` and `AspifSymbolicPrinter` in that script should implement the interface:
* https://potassco.org/clingo/python-api/5.6/clingo/backend.html#clingo.backend.Observer

Consider the following example:

In [1]:
%%file example.lp
dom(1..2).

a(X) :- dom(X). 
b(X) :- not a(X), dom(X).

{ c(X) } :- dom(X).
d(X) :- not c(X), dom(X).

  e(X); f(X); g(X)   :- c(X), d(X), dom(X).
{ e(X); f(X); g(X) } :- c(X), d(X), dom(X).

h :- 1 { e(X); f(X) }.
i :- 1 { e(X); f(X) } 1.

:- h, i.

#show (e(X),f(X),g(X)) : e(X), f(X), g(X).

#show c/1. 
#show d/1. 
#show e/1. 
#show f/1. 
#show g/1.

#minimize{ 1@3,X : e(X); 2@3,X : f(X); 3,X : g(X)}.

#external j. [true]

#heuristic e(X) : c(X). [1,level]

#project e/1.

Writing example.lp


We can ground the program using option `--mode=gringo`, and we obtain a ground logic program in the aspif format:

In [2]:
! clingo --mode=gringo example.lp

asp 1 0 0
1 0 1 1 0 0
1 0 1 2 0 0
1 1 1 3 0 0
1 1 1 4 0 0
1 0 1 5 0 1 -3
1 0 1 6 0 1 -4
1 0 3 7 8 9 0 2 5 3
1 0 3 10 11 12 0 2 6 4
1 1 3 7 8 9 0 2 5 3
1 1 3 10 11 12 0 2 6 4
3 1 7
3 1 10
7 0 7 1 0 1 3
7 0 10 1 0 1 4
5 13 1
1 0 1 14 1 1 4 7 1 10 1 8 1 11 1
1 0 1 15 1 2 4 7 1 10 1 8 1 11 1
1 0 1 16 0 2 14 -15
1 0 1 17 0 1 16
1 0 1 18 1 1 4 7 1 10 1 8 1 11 1
1 0 1 19 0 1 18
1 0 0 0 2 17 19
1 0 1 20 0 0
1 0 1 21 0 0
2 0 2 9 3 12 3
2 3 4 8 2 11 2 7 1 10 1
4 4 c(1) 1 3
4 4 c(2) 1 4
4 4 d(1) 1 5
4 4 d(2) 1 6
4 4 e(1) 1 7
4 4 e(2) 1 10
4 4 f(1) 1 8
4 4 f(2) 1 11
4 4 g(1) 1 9
4 4 g(2) 1 12
1 0 1 22 0 3 7 8 9
4 16 (e(1),f(1),g(1)) 1 22
1 0 1 23 0 3 10 11 12
4 16 (e(2),f(2),g(2)) 1 23
0


Using our script, we should obtain a more readable version of this program.

The output of:

In [ ]:
! python aspif-pretty-printer.py example.lp

should look like this:

```
1.
2.
{ 3 }.
{ 4 }.
5 :- -3.
6 :- -4.
7;8;9 :- 5, 3.
10;11;12 :- 6, 4.
{ 7; 8; 9 } :- 5, 3.
{ 10; 11; 12 } :- 6, 4.
#project 7.
#project 10.
#heuristic 7 : 3. [1@0,level]
#heuristic 10 : 4. [1@0,level]
#external 13. [true]
14 :- 1 { 1:7; 1:10; 1:8; 1:11 }.
15 :- 2 { 1:7; 1:10; 1:8; 1:11 }.
16 :- 14, -15.
17 :- 16.
18 :- 1 { 1:7; 1:10; 1:8; 1:11 }.
19 :- 18.
:- 17, 19.
20.
21.
#minimize{ 3@0:9, 3@0:12 }.
#minimize{ 2@3:8, 2@3:11, 1@3:7, 1@3:10 }.
#show c(1) : 3.
#show c(2) : 4.
#show d(1) : 5.
#show d(2) : 6.
#show e(1) : 7.
#show e(2) : 10.
#show f(1) : 8.
#show f(2) : 11.
#show g(1) : 9.
#show g(2) : 12.
22 :- 7, 8, 9.
#show (e(1),f(1),g(1)) : 22.
23 :- 10, 11, 12.
#show (e(2),f(2),g(2)) : 23.
```

Option `--text` should use the shown atoms to replace their corresponding literals in the program.

The result of:

In [ ]:
! python aspif-pretty-printer.py --text example.lp

should look like this:

```
1.
2.
{ c(1) }.
{ c(2) }.
d(1) :- not c(1).
d(2) :- not c(2).
e(1);f(1);g(1) :- d(1), c(1).
e(2);f(2);g(2) :- d(2), c(2).
{ e(1); f(1); g(1) } :- d(1), c(1).
{ e(2); f(2); g(2) } :- d(2), c(2).
16 :- 14, -15.
17 :- 16.
19 :- 18.
:- 17, 19.
20.
21.
22 :- e(1), f(1), g(1).
23 :- e(2), f(2), g(2).
14 :- 1 { 1:e(1); 1:e(2); 1:f(1); 1:f(2) }.
15 :- 2 { 1:e(1); 1:e(2); 1:f(1); 1:f(2) }.
18 :- 1 { 1:e(1); 1:e(2); 1:f(1); 1:f(2) }.
#external 13. [true]
#heuristic e(1) : c(1). [1@0,level]
#heuristic e(2) : c(2). [1@0,level]
#minimize{ 3@0:g(1), 3@0:g(2) }.
#minimize{ 2@3:f(1), 2@3:f(2), 1@3:e(1), 1@3:e(2) }.
#show (e(1),f(1),g(1)) : 22.
#show (e(2),f(2),g(2)) : 23.
#project e(1).
#project e(2).
```

Observe how numbers `3` to `12` have been replaced by their corresponding symbolic atoms.